In [1]:
!pip3 install spacy -q
!python3 -m spacy download en -q
!python3 -m spacy download en_core_news_md -q

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')

✘ No compatible package found for 'en_core_news_md' (spaCy v3.2.4)



In [2]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from urllib.parse import urlparse
import pickle
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
fn = "df_st_ps.pk"
df = pd.read_pickle(fn)
df.sample(5)

,labels,text,text_stem
19,0,"Almost 100,000 people left Puerto Rico last ye...","[almost, 100,000, peopl, left, puerto, rico, l..."
7044,1,Says Floridas pension plan faces the same long...,"[say, florida, pension, plan, face, the, same,..."
5961,1,"These steps (a middle-class tax cut, national ...","[these, step, (a, middle-class, tax, cut,, nat..."
10113,1,The majority last quarter of Sherrod Browns ca...,"[the, major, last, quarter, of, sherrod, brown..."
2216,0,The U.S. Department of Defense spends $80 mill...,"[the, u.s., depart, of, defens, spend, $80, mi..."


In [4]:
 df["text_stem"].map(lambda l: " ".join(l))

0        say the anni list polit group support third-tr...
1        when did the declin of coal start? it start wh...
2        hillari clinton agre with john mccain "bi vote...
3        health care reform legisl is like to mandat fr...
4        the econom turnaround start at the end of my t...
                               ...                        
10234    under the rule of the suprem court, ani lobbyi...
10235    there are a larger number of shark attack in f...
10236    democrat have now becom the parti of the [atla...
10237    say an altern to social secur that oper in gal...
10238    on lift the u.s. cuban embargo and allow trave...
Name: text_stem, Length: 9399, dtype: object

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.model_selection import cross_val_score, GridSearchCV

Y = df["labels"]
X = df["text_stem"].map(lambda l: " ".join(l))

pipeline = Pipeline(
    [
        ("CV", CountVectorizer()), 
        ("mnb", MultinomialNB(class_prior=None))
    ]
)

param_grid = {
    
    "CV__ngram_range": [(1,1),(1,2)],
    "CV__max_df": [0.95,0.90,0.85,0.80,0.75,0.70,0.65,0.60,0.55,0.50, 0.45, 0.40],
    "CV__min_df": [0.0001, 0.00025, 0.0005, 0.001, 0.0025, 0.005],
    "CV__binary": [True,False],
    
    
    
    
    "mnb__alpha": [1, 4, 16, 64, 128, 256, 512],
    "mnb__fit_prior": [False]
    
    
}

SCORING = "f1"


grid = GridSearchCV(
    pipeline,
    n_jobs=-1,
    param_grid=param_grid,
    verbose=5,
    # réentrainer le meilleur modèle sur toutes les données
    refit=True,
    scoring=SCORING,
)





grid.fit(X, Y)

Fitting 5 folds for each of 2016 candidates, totalling 10080 fits
[CV 3/5] END CV__binary=True, CV__max_df=0.95, CV__min_df=0.0001, CV__ngram_range=(1, 1), mnb__alpha=1, mnb__fit_prior=False;, score=0.665 total time=   1.0s
[CV 1/5] END CV__binary=True, CV__max_df=0.95, CV__min_df=0.0001, CV__ngram_range=(1, 1), mnb__alpha=4, mnb__fit_prior=False;, score=0.744 total time=   0.9s
[CV 4/5] END CV__binary=True, CV__max_df=0.95, CV__min_df=0.0001, CV__ngram_range=(1, 1), mnb__alpha=4, mnb__fit_prior=False;, score=0.746 total time=   1.1s
[CV 2/5] END CV__binary=True, CV__max_df=0.95, CV__min_df=0.0001, CV__ngram_range=(1, 1), mnb__alpha=16, mnb__fit_prior=False;, score=0.760 total time=   1.1s
[CV 5/5] END CV__binary=True, CV__max_df=0.95, CV__min_df=0.0001, CV__ngram_range=(1, 1), mnb__alpha=16, mnb__fit_prior=False;, score=0.759 total time=   1.2s
[CV 5/5] END CV__binary=True, CV__max_df=0.95, CV__min_df=0.0001, CV__ngram_range=(1, 1), mnb__alpha=64, mnb__fit_prior=False;, score=0.760 to

GridSearchCV(estimator=Pipeline(steps=[('CV', CountVectorizer()),
                                       ('mnb', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'CV__binary': [True, False],
                         'CV__max_df': [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65,
                                        0.6, 0.55, 0.5, 0.45, 0.4],
                         'CV__min_df': [0.0001, 0.00025, 0.0005, 0.001, 0.0025,
                                        0.005],
                         'CV__ngram_range': [(1, 1), (1, 2)],
                         'mnb__alpha': [1, 4, 16, 64, 128, 256, 512],
                         'mnb__fit_prior': [False]},
             scoring='f1', verbose=5)

[CV 4/5] END CV__binary=False, CV__max_df=0.4, CV__min_df=0.005, CV__ngram_range=(1, 1), mnb__alpha=4, mnb__fit_prior=False;, score=0.640 total time=   0.7s
[CV 1/5] END CV__binary=False, CV__max_df=0.4, CV__min_df=0.005, CV__ngram_range=(1, 1), mnb__alpha=16, mnb__fit_prior=False;, score=0.676 total time=   1.2s
[CV 4/5] END CV__binary=False, CV__max_df=0.4, CV__min_df=0.005, CV__ngram_range=(1, 1), mnb__alpha=16, mnb__fit_prior=False;, score=0.657 total time=   1.1s
[CV 3/5] END CV__binary=False, CV__max_df=0.4, CV__min_df=0.005, CV__ngram_range=(1, 1), mnb__alpha=64, mnb__fit_prior=False;, score=0.706 total time=   1.2s
[CV 1/5] END CV__binary=False, CV__max_df=0.4, CV__min_df=0.005, CV__ngram_range=(1, 1), mnb__alpha=128, mnb__fit_prior=False;, score=0.746 total time=   1.3s
[CV 4/5] END CV__binary=False, CV__max_df=0.4, CV__min_df=0.005, CV__ngram_range=(1, 1), mnb__alpha=128, mnb__fit_prior=False;, score=0.753 total time=   1.4s
[CV 3/5] END CV__binary=False, CV__max_df=0.4, CV__

In [7]:
grid.best_estimator_

Pipeline(steps=[('CV',
                 CountVectorizer(binary=True, max_df=0.95, min_df=0.00025,
                                 ngram_range=(1, 2))),
                ('mnb', MultinomialNB(alpha=16, fit_prior=False))])

In [8]:
grid.best_score_

0.7602742084463755

In [10]:
len(grid.best_estimator_['CV'].vocabulary_)

15010

In [11]:
import joblib
joblib.dump(grid, '4_ml_nb_st_ps_cv_best.pkl') 

['4_ml_nb_st_ps_cv_best.pkl']